In [32]:
%load_ext autoreload
%autoreload 2

# Load spacy tokenizer
from spacy.lang.es import Spanish

nlp = Spanish()

spacy_tokenizer = nlp.tokenizer

list(spacy_tokenizer("Venga que esto es una risa tío"))

[Venga, que, esto, es, una, risa, tío]

In [33]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_name = "pysentimiento/robertuito-ner"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

In [43]:
sentences = [
    "abran paso al mejor de todos los tiempos, Leonel Andrés messi cuccittini",
    "sos el mejor leo",
]


words = [
    [token.text for token in spacy_tokenizer(sentence)] for sentence in sentences
]
    



In [44]:
import torch 


inputs = tokenizer(words, is_split_into_words=True, padding=True)

model_inputs = {k: torch.tensor(v) for k, v in inputs.items()}

outs = model(**model_inputs)

In [49]:
id2label = model.config.id2label

outputs = torch.argmax(outs.logits, dim=2)



In [50]:
outputs

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 11, 12, 12, 12, 12, 12, 12,  0],
        [ 0,  0,  0,  0, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [59]:
labels = []

for i, (sentence, output) in enumerate(zip(words, outputs)):

    sentence_labels = [None for _ in sentence]
    print(sentence)
    print(output)
    word_ids = inputs.word_ids(i)
    print(word_ids)

    for word_id, label in zip(word_ids, output):
        if word_id is not None and sentence_labels[word_id] is None:
                sentence_labels[word_id] = id2label[label.item()]

    print(list(zip(sentence, sentence_labels)))
    # for word, label in zip(sentence, output):
    #     print(word, id2label[label.item()])

['abran', 'paso', 'al', 'mejor', 'de', 'todos', 'los', 'tiempos', ',', 'Leonel', 'Andrés', 'messi', 'cuccittini']
tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 11, 12, 12, 12, 12, 12, 12,  0])
[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 12, 12, 12, None]
[('abran', 'O'), ('paso', 'O'), ('al', 'O'), ('mejor', 'O'), ('de', 'O'), ('todos', 'O'), ('los', 'O'), ('tiempos', 'O'), (',', 'O'), ('Leonel', 'B-PER'), ('Andrés', 'I-PER'), ('messi', 'I-PER'), ('cuccittini', 'I-PER')]
['sos', 'el', 'mejor', 'leo']
tensor([ 0,  0,  0,  0, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
[None, 0, 1, 2, 3, None, None, None, None, None, None, None, None, None, None, None, None, None]
[('sos', 'O'), ('el', 'O'), ('mejor', 'O'), ('leo', 'B-PER')]


In [40]:


for i, sent in enumerate(words):
    for i, label in enumerate(outputs):
        word_id = inputs.word_ids(i)[i]

        if word_id is not None and labels[word_id] is None:
            labels[word_id] = label

TypeError: 'NoneType' object is not subscriptable